In [ ]:
#install packages if not present
install.packages('dplyr')
install.packages('ggplot2')

In [ ]:
library(dplyr)
library(ggplot2)

In [ ]:
## Non-gap filled
nft_url <- 'https://raw.githubusercontent.com/madeleineernst/Metabolomics_SummerSchool_2022/main/data/MZmine/Xenobiotic_Metabolism_ChemProp2_NonGapFilled_QuantTable.csv'
## Gap filled
ft_url <- 'https://raw.githubusercontent.com/madeleineernst/Metabolomics_SummerSchool_2022/main/data/MZmine/Xenobiotic_Metabolism_ChemProp2_GapFilled_QuantTable.csv'
md_url <- 'https://raw.githubusercontent.com/madeleineernst/Metabolomics_SummerSchool_2022/main/data/Xenobiotic_Metabolism_metadata.txt'

In [ ]:
nft <- read.csv(nft_url, header = T, check.names = F)
ft <- read.csv(ft_url, header = T, check.names = F)
md <- read.csv(md_url, header = T, check.names = F, sep = '\t')

In [ ]:
head(ft)

In [ ]:
head(nft)

In [ ]:
head(md)

In [ ]:
#Removing Peak area extensions
colnames(ft) <- gsub(' Peak area','',colnames(ft))
colnames(nft) <- gsub(' Peak area','',colnames(nft))
md$filename <- gsub(' Peak area','',md$filename)

#Removing if any NA columns present in the md file
md <- md[,colSums(is.na(md))<nrow(md)]

#Changing the row names of the files
rownames(md) <- md$filename
rownames(ft) <- paste(ft$'row ID',round(ft$'row m/z',digits = 3),round(ft$'row retention time',digits = 3), sep = '_')
rownames(nft) <- paste(nft$'row ID',round(nft$'row m/z',digits = 3),round(nft$'row retention time',digits = 3), sep = '_')

#Picking only the files with column names containing 'mzML'
ft <- ft[,grep('mzML',colnames(ft))]
nft <- nft[,grep('mzML',colnames(nft))]

# Converting replicate attributes into factors (categorical data)
md$ATTRIBUTE_replicates <- as.factor(md$ATTRIBUTE_replicates)

In [ ]:
head(nft)
dim(nft)

In [ ]:
head(ft)
dim(ft)

### Creating a function named FrequencyPlot:  
The below function takes in the two input datatables: gapfilled and non-gapfilled, calculates the frequency distribution of the data in the order of 10 and produces a grouped barplot showing the distribution as output. The frequency plot shows where the features are present in higher number.

In [ ]:
options(repr.plot.width=5, repr.plot.height=3) #'global' settings for plot size in the output cell

FrequencyPlot <- function(x1,x2){
  
  bins <- c(-1,0,(1 * 10^(seq(0,10,1)))) #creating bins from -1 to 10^10 using sequence function: seq()
  
  scores_x1 <- cut(as.matrix(x1),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10')) #cut function store the data into the appropriate bins
  Table_x1<-transform(table(scores_x1)) #transform function convert the tables into column format: easy for visualization
  
  # Repeating the same steps for x2
  scores_x2 <- cut(as.matrix(x2),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10'))
  Table_x2<-transform(table(scores_x2))
  
  arg1 <- deparse(substitute(x1))
  arg2 <-deparse(substitute(x2))
  
  data_plot <- as.data.frame(c(Table_x1$Freq,Table_x2$Freq))
  colnames(data_plot) <- "Freq"
  data_plot$Condition <- c(rep(arg1,12),rep(arg2,12))
  data_plot$Range_bins <- rep(Table_x1$scores_x1,2)
  data_plot$Log_Freq <- log(data_plot$Freq+1) #Log scaling the frequency values
  
  ## GGPLOT2
   BarPlot <- ggplot(data_plot, aes(Range_bins, Log_Freq, fill = Condition)) + 
    geom_bar(stat="identity", position = "dodge", width=0.4) + 
    scale_fill_brewer(palette = "Set1") +
    ggtitle(label="Frequency plot") +
    xlab("Range") + ylab("(Log)Frequency") + labs(fill = "Data Type") + 
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +   # setting the angle for the x label
    theme(axis.text.y = element_text(angle = 45, vjust = 0.5, hjust=1)) +   # setting the angle for the y label
    theme(plot.title = element_text(hjust = 0.5)) # centering the plot title
  
  print(BarPlot)
}  

**Imputation:** Replacing zeros from the gapfilled quant-table with the minimum value of non-gapfilled table

In [ ]:
GapFilled <- ft
NotGapFilled <- nft

if(readline('Do you want to perform Imputation - Y/N:')=='Y'){
    
    plot<- FrequencyPlot(GapFilled,NotGapFilled)
    
    Arg1 = plot$data$Condition[1]
    Arg2 = plot$data$Condition[13]
    plotData_New <- subset(plot$data,plot$data$Freq!=0 & plot$data$Range_bins !=0) # accessing the datatable of plot and subsetting with the condition: Eliminating the Range (or bin) 0 and Ranges with zero frequencies 
    First_val_temp <- aggregate(plotData_New$Freq, by=list(plotData_New$Condition), FUN=first) #getting the first appearing value of this new plot datatable
    First_val <- plotData_New[plotData_New$Freq %in% c(First_val_temp$x[1],First_val_temp$x[2]),] # Subsetting the rows in the plotData_New that has the first appearing values
  
    print(paste0("The Range with a minimum value greater than 0 for ",Arg1,":", First_val$Range_bins[1]))
    print(paste0("The Range with a minimum value greater than 0 for ",Arg2,":", First_val$Range_bins[2]))
    
    RawLOD <- round(min(NotGapFilled[NotGapFilled!=min(NotGapFilled)])) # getting the 2nd minimum value of non-gap filled data. (The first minimum value in the data table is usually zero)
    
    print(paste0("The minimum value in the Non-gap filled data other than 0 : ",RawLOD))
    GapFilled[GapFilled==0 | GapFilled<RawLOD] <- RawLOD # Replacing zeros in the gap-filled file as well as values<RawLOD with RawLOD
    RawLOD_Table <- GapFilled
    #write.csv(RawLOD_Table, file=paste0('Quant_Table_filled_with_MinValue_',RawLOD,'_NotGapFilled','.csv'),row.names =FALSE) 
    input_data <- GapFilled
} else input_data <- GapFilled

## Blank Removal:
Now, we have the resultant input_data, which is 'ft' filled with 3766 instead of 0s. For the next step, Blank removal, we need to split the data as control and samples.
- The control here is the sample without treatment. 
- Samples are biological replicates with treatment and we have two sets of data in our file, B.sub and E.coli.

In [ ]:
head(input_data)

In [ ]:
Ctrl <-  input_data[,grep('_C',colnames(input_data))]
Samples <- input_data[,-grep('_C',colnames(input_data))]

In [ ]:
if(readline('Do you want to perform Blank Removal- Y/N:')=='Y'){
    
    #When cutoff is low, more noise (or background) detected; With higher cutoff, less background detected, thus more features observed
    Cutoff <- as.numeric(readline('Enter Cutoff value between 0.1 & 1:')) # (i.e. 10% - 100%). Ideal cutoff range: 0.1-0.3
    
    #Getting mean for every feature in Ctrl and Samples
    Avg_ctrl <- rowMeans(Ctrl, na.rm= FALSE, dims = 1) # set na.rm = FALSE to check if there are NA values. When set as TRUE, NA values are changed to 0
    Avg_samples <- rowMeans(Samples, na.rm= FALSE, dims = 1)
    
    #Getting the ratio of Ctrl vs Sample
    Ratio_Ctrl_Sample <- (Avg_ctrl+1)/(Avg_samples+1)
    
    # Creating a bin with 1s when the ratio>Cutoff, else put 0s
    Bg_bin <- ifelse(Ratio_Ctrl_Sample > Cutoff, 1, 0 )


    # Checking if there are any NA values present. Having NA values in the 4 variables will affect the final dataset to be created
    temp_NA_Count <-cbind(Avg_ctrl ,Avg_samples,Ratio_Ctrl_Sample,Bg_bin)
    cat('NA values are present in:',names(which(colSums(is.na(temp_NA_Count))>0))) #prints the variable name if it has NA values

     #Calculating the number of background features and features present
    Bg_Features <- sum(Bg_bin ==1,na.rm = TRUE)
    No_of_Features <- nrow(input_data) - Bg_Features
    
    print(paste("No.of Background or noise features:",Bg_Features))
    print(paste("No.of features after excluding noise:",No_of_Features)) 

    input_data1 <- cbind(as.matrix(input_data),Bg_bin)    
    plot_CtrlSample <- FrequencyPlot(Samples,Ctrl)
}

In [ ]:
head(Ctrl)
head(Samples)

Previously we have replaced the zeros in our ft(gapfilled) with the minimum value in nft(non gapfilled). Ex: 3766

The frequency plot shows where the features are present in higher number. We can only use ft and see the frquency distribution of its features with the frequency plot.
For ex: if until 1E2 has no or really less features, the goal is to exclude until that range and consider only values from 1E3 range. Thus 1E3 will be taken as Cutoff_LOD (Limit of Detection). This value will eventually become the 'new zero'.  

The following step asks if the imputation was already performed, if so, it takes that value as the Cutoff_LOD, else, we get to specify our Cutoff_LOD based on the frequency plot.
Once we have our Cutoff_LOD,  
--- We create  a temporary dataset checking all the feature intensites of our sample (only the sample, without control) and check it against the Cutoff_LOD. If it is less than the Cutoff_LOD, we replace it with Cutoff_LOD. Thus our sample data, for example, is with a bunch of 1000s (if our Cutoff_LOD=1000) instead of zeros  
--- Then, we create a Final dataset using the temporary dataset. Here, we try to see if each feature from all samples is noise or not. If it noise, we replace it with Cutoff_LOD as well, else we keep the info from the temporary dataset as such. 

In [ ]:
Cutoff_LOD <-ifelse(readline("Was Imputation step already performed? Y/N :")=="Y",RawLOD,as.numeric(readline("Enter your Cutoff LOD here:")))  #Enter the LOD value as seen in the frequency plot
temp_matrix <- c()
for (i in 1:ncol(Samples)){ 
    
    #Replacing the Sample intensities with Cutoff_LOD, if they are lower than Cutoff_LOD
    x <- ifelse(Samples[,i] > Cutoff_LOD, Samples[,i],Cutoff_LOD)
    temp_matrix <- cbind(temp_matrix,as.matrix(x))
}
colnames(temp_matrix) <- colnames(Samples)
  
Final_matrix <-c()
for (i in 1:ncol(temp_matrix)){
    
     #Replacing the feature row with Cutoff_LOD if its Bg_bin value is 1, indicating it as noise, else retain the temporary dataset as such.
    x <-ifelse(input_data1[,ncol(input_data1)] ==1, Cutoff_LOD, temp_matrix[,i])
    Final_matrix <-cbind(Final_matrix,x)
}
colnames(Final_matrix) <- colnames(Samples)
#write.csv(Final_matrix,file=paste0('Processed_Quant_Table_filled_with_Value_',Cutoff_LOD ,'.csv'),row.names =TRUE)


In [ ]:
head(Final_matrix)

In [ ]:
Final_matrix<-Final_matrix[rowMeans(Final_matrix)!= Cutoff_LOD,]  #removing all the rows with only cutoff values

## Normalization:
The following code performs sample-centric (column-wise) normalisation:

In [ ]:
if (readline("Do you want to perform Normalization: Y/N:") == 'Y'){
    input_data <- Final_matrix
    
    #Getting column-wise sums of the input-data
    sample_sum <- colSums(input_data, na.rm= TRUE, dims = 1)
    
    #Dividing each element of a particular column with its column sum
    Normalized_data <- c()
    for (i in 1:ncol(input_data)){
        x <- input_data[,i] / sample_sum[i]
        Normalized_data <- cbind(Normalized_data, x)
    }
    colnames(Normalized_data) <- names(sample_sum)

} else return(Final_matrix)
  
print(paste('No.of NA values in Normalized data:',sum(is.na(Normalized_data)== TRUE)))
#write.csv(Normalized_data,file='Normalised_Quant_table.csv',row.names =TRUE)
    